# OpenStreetMap Data Case Study [Seoul, South Korea]
---

## 1. Introduction
---

### 1) Map Area
    - Seoul, South Korea (central part)  
      
    https://mapzen.com/data/metro-extracts/metro/seoul_south-korea/  
    
    This is my hometown, Seoul, the capital of South Korea. I decided to take a deep look into the open street map dataset and figure out what I can contribute. Originally, The boundary of dataset was wider than current dataset, which was containg mixed area of Seoul and non-Seoul. I tried to reduce the boundary so that it can contain the central part of the Seoul. Consequently, the size of dataset reduced from 337.1MB to 79.5MB.  
   
    - Changes  

    From  
        minlat="37.019"   
        minlon="126.064"   
        maxlat="37.98"   
        maxlon="127.525"  
  
    To  
        minlat="37.4694085"   
        minlon="126.8845367"   
        maxlat="37.6115117"   
        maxlon="127.1063232"  



  
---

### 2) Import Data

    - Libraries  

    Imported libraries(csv, codec, pprint, re, cElementTree, cerberus) needed and defined schema to check validity.
    
    - Schema  
    
    If the element top level tag is "node": The dictionary returned have the format {"node": .., "node_tags": ...}. The "node" field hold a dictionary of the following top level node attributes: id / user / uid / version / lat / lon / timestamp / changeset and all other attributes are ignored

    The "node_tags" field should hold a list of dictionaries, one per secondary tag. Secondary tags are
    child tags of node which have the tag name/type: "tag". Each dictionary should have the following
    fields from the secondary tag attributes:
    id: the top level node id attribute value
    key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
    value: the tag "v" attribute value
    type: either the characters before the colon in the tag "k" value or "regular" if a colon is not present.

    If the element top level tag is "way": the dictionary have the format {"way": ..., "way_tags": ..., "way_nodes": ...}. The "way" field hold a dictionary of the following top level way attributes: id / user / uid / version / timestamp / changeset and all other attributes can be ignored

    The "way_tags" field hold a list of dictionaries, following the exact same rules as for "node_tags".

    Additionally, the dictionary should have a field "way_nodes". "way_nodes" hold a list of dictionaries, one for each nd child tag.  Each dictionary should have the fields:
    id: the top level element (way) id
    node_id: the ref attribute value of the nd tag
    position: the index starting at 0 of the nd tag i.e. what order the nd tag appears within the way element
    
---

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import cerberus

OSM_PATH = "central-seoul_south-korea.osm"
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

SCHEMA = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}


# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


    - Helper Functions  
    
     'get_element' function parse osm file with tags named 'node' and 'way'. 'validate_element' check if there are mismatches in fields created and schema. 'UnicodeDictWriter' encode Korean language into unicode and write data on file with unicode so that Korean can be used in csv files.

In [2]:
# ================================================== #
#               Helper Functions                     #
# ================================================== #

def get_element(osm_file, tags=('node', 'way')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)



### 3) Keys of tag
---

In [3]:
from collections import Counter
node_key = []
way_key = []
for element in get_element('central-seoul_south-korea.osm'):
    if element.tag == 'node':
        for tag in element.iter('tag'):
            node_key.append(tag.attrib['k'])
    elif element.tag == 'way': 
        for tag in element.iter('tag'):
            way_key.append(tag.attrib['k'])
node_count = Counter(node_key)
way_count = Counter(way_key)
print node_count
print way_count

Counter({'name': 14278, 'name:en': 11185, 'name:ko': 9346, 'name:ko_rm': 8978, 'amenity': 5697, 'source': 5319, 'ncat': 5200, 'highway': 4418, 'route_ref': 3637, 'railway': 1567, 'tourism': 1082, 'shop': 1066, 'name:ja': 1051, 'addr:street': 642, 'addr:housenumber': 620, 'building': 513, 'place': 406, 'addr:postcode': 401, 'addr:city': 381, 'entrance': 380, 'cuisine': 376, 'public_transport': 293, 'subway': 257, 'barrier': 245, 'historic': 227, 'ref': 211, 'natural': 148, 'website': 147, 'phone': 144, 'opening_hours': 143, 'operator': 142, 'atm': 138, 'level': 133, 'leisure': 115, 'crossing': 107, 'station': 106, 'internet_access': 101, 'direction': 98, 'information': 96, 'wheelchair': 95, 'access': 92, 'junction': 90, 'layer': 78, 'material': 77, 'artwork_type': 75, 'foot': 72, 'board_type': 72, 'religion': 71, 'landuse': 66, 'description:en': 62, 'fixme': 59, 'man_made': 59, 'wikipedia': 58, 'noexit': 57, 'ele': 56, 'note': 54, 'statue': 53, 'name:zh': 52, 'network': 47, 'smoking': 4

## 2. Cleaning Data
---


### 0) Problems encountered in the map  
---  
    Most of data were valid except attribute 'k' and 'v' of tags in nodes and ways. There were few problems in both key and value sides.
    
    Key Sides
    - Names with more than 1 languages('name','name:en','name':jp etc). They are separated by ":" which should not be sort as 'kind'.  
    - There are keys with more than one ':'.
    - Too many kinds of "k" most of which are inaccurate or inconsistant.  
    - Valuse with problematic characters    

    Value Sides
    - Even though two kinds of postcodes are being used in Korea, still there were “Incorrect” postcodes.


### 1) Sorting out problematic characters  

    I wanted to deal some of the problems before data from osm are imported to SQL. I sorted out problematic characters by defining 'PROBLEMCHARS'.

### 2) Dealing with names  

    I tried to deal keys with string 'name' as exception for assigning kinds. However, if there is a word before name (For example like "bridge:name" or "bridge:name:en"), I separate them and assigned to 'type'. 

### 3) Assigning 'kind'  

     For the rest, I assigned 'kind' and shaped them into CSV file.

### 4) Dealing with inconsistant 'k' value

    I used Counter function to figure out usage of keys. I've discovered there are some outliers with too few usage. I tried to limit the candidate of 'k' with more than 10 elements. 
--- 

In [4]:
freq_node = [key for key in node_count if node_count[key] >10] 
freq_way = [key for key in node_count if way_count[key] >10] 
print "Node: "
print freq_node
print "Way: "
print freq_way

Node: 
['is_in', 'addr:street', 'level', 'ref', 'level:ref', 'subway', 'bicycle', 'access', 'source:bridge', 'camera:direction', 'amenity', 'alt_name', 'fee', 'start_date', 'entrance', 'phone', 'species', 'information', 'name:fr', 'description', 'natural', 'wheelchair', 'office', 'old_operator', 'addr:housenumber', 'camera:mount', 'covered', 'image', 'junction', 'material', 'camera:type', 'foot', 'tourism', 'payment:bitcoin', 'fixme', 'name', 'designation', 'crossing', 'network', 'name:zh', 'highway', 'barrier', 'ncat:bridge', 'noexit', 'atm', 'place', 'addr:housename', 'station', 'building', 'bridge:name:ko', 'aeroway', 'landuse', 'bridge', 'bridge:name:ko_rm', 'sport', 'note', 'tank:type', 'shop', 'name:ko', 'indoor', 'leisure', 'name:el', 'name:en', 'addr:postcode', 'internet_access:fee', 'public_transport', 'leaf_cycle', 'cannon:type', 'historic:period', 'bridge:name:en', 'route_ref', 'parking', 'capacity', 'historic:civilization', 'wikipedia', 'mapillary', 'ele', 'denomination', '

In [5]:
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    # Deal with node;
    if element.tag == 'node':
        for i in node_attr_fields:
            node_attribs[i] = element.attrib[i]
    # Deal with node - tag;
        tags = []  
        for tag in element.iter('tag'):
        # Sort out problematic characters / Limit the 'v' candidates
            if (problem_chars.search(tag.attrib['k'])) or (tag.attrib['v'] not in freq_node):
                pass
            else:
                tagdic = {}
                tagdic['id'] = element.attrib['id']
                if ":" in tag.attrib['k']:
        # Keep the name
                    if "name" in tag.attrib['k']:
                        if ":name" in tag.attrib['k']:
        # Assign types
                            k = list(tag.attrib['k'].split(":"))
                            if len(k) == 2:
                                tagdic['key'] = k[1]
                                tagdic['type'] = k[0]
                                tagdic['value'] = tag.attrib['v']
                                tags.append(tagdic)
                            else:
                                tagdic['key'] = k[1]+":"+k[2]
                                tagdic['type'] = k[0]
                                tagdic['value'] = tag.attrib['v']
                                tags.append(tagdic)                                
                        else:
                            tagdic['key'] = tag.attrib['k']
                            tagdic['type'] = 'regular'
                            tagdic['value'] = tag.attrib['v']
                            tags.append(tagdic)
                    else: 
                        k = list(tag.attrib['k'].split(":"))
                        if len(k) == 2:
                            tagdic['key'] = k[1]
                            tagdic['type'] = k[0]
                            tagdic['value'] = tag.attrib['v']
                            tags.append(tagdic)
                        else:
                            tagdic['key'] = k[1]+":"+k[2]
                            tagdic['type'] = k[0]
                            tagdic['value'] = tag.attrib['v']
                            tags.append(tagdic)
                else:
                    tagdic['key'] = tag.attrib['k']
                    tagdic['type'] = 'regular'
                    tagdic['value'] = tag.attrib['v']
                    tags.append(tagdic)
        return {'node': node_attribs, 'node_tags': tags}
    # Deal with way;
    elif element.tag == 'way':
        for i in way_attr_fields:
            way_attribs[i] = element.attrib[i]
    # Deal with way - nd;
        count = 0
        for node in element.iter("nd"):
            nd = {}
            nd['id'] = element.attrib['id']
            nd['node_id'] = node.attrib['ref']
            nd['position'] = count
            way_nodes.append(nd)
            count += 1
    # Deal with way - tag;
        tags = []  
        for tag in element.iter('tag'):
        # Sort out problematic characters / Limit the 'v' candidates
            if (problem_chars.search(tag.attrib['k'])) or (tag.attrib['v'] not in freq_way):
                pass
            else:
                tagdic = {}
                tagdic['id'] = element.attrib['id']
                if ":" in tag.attrib['k']:
        # Keep the name
                    if "name" in tag.attrib['k']:
                        if ":name" in tag.attrib['k']:
        # Assign types
                            k = list(tag.attrib['k'].split(":"))
                            if len(k) == 2:
                                tagdic['key'] = k[1]
                                tagdic['type'] = k[0]
                                tagdic['value'] = tag.attrib['v']
                                tags.append(tagdic)
                            else:
                                tagdic['key'] = k[1]+":"+k[2]
                                tagdic['type'] = k[0]
                                tagdic['value'] = tag.attrib['v']
                                tags.append(tagdic)                                
                        else:
                            tagdic['key'] = tag.attrib['k']
                            tagdic['type'] = 'regular'
                            tagdic['value'] = tag.attrib['v']
                            tags.append(tagdic)
                    else: 
                        k = list(tag.attrib['k'].split(":"))
                        if len(k) == 2:
                            tagdic['key'] = k[1]
                            tagdic['type'] = k[0]
                            tagdic['value'] = tag.attrib['v']
                            tags.append(tagdic)
                        else:
                            tagdic['key'] = k[1]+":"+k[2]
                            tagdic['type'] = k[0]
                            tagdic['value'] = tag.attrib['v']
                            tags.append(tagdic)
                else:
                    tagdic['key'] = tag.attrib['k']
                    tagdic['type'] = 'regular'
                    tagdic['value'] = tag.attrib['v']
                    tags.append(tagdic)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

    - Write on CSV file
    File named 
        nodes.csv,
        nodes_tags.csv,
        ways.csv,
        ways_nodes.csv,
        ways_tags.csv.

In [6]:
# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()
        count = 0
        for element in get_element(file_in, tags=('node', 'way')):
            if count%1 == 0:
                el = shape_element(element)
                if el:
                    if validate is True:
                        validate_element(el, validator)

                    if element.tag == 'node':
                        nodes_writer.writerow(el['node'])
                        node_tags_writer.writerows(el['node_tags'])
                    elif element.tag == 'way':
                        ways_writer.writerow(el['way'])
                        way_nodes_writer.writerows(el['way_nodes'])
                        way_tags_writer.writerows(el['way_tags'])
            count += 1


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

### 5) Updating wrong postcodes
---

    - Import to SQL
    1.
    sqlite3 osm.db
    2.
    CREATE TABLE nodes (
        id INTEGER PRIMARY KEY NOT NULL,
        lat REAL,
        lon REAL,
        user TEXT,
        uid INTEGER,
        version INTEGER,
        changeset INTEGER,
        timestamp TEXT
    );

    CREATE TABLE nodes_tags (
        id INTEGER,
        key TEXT,
        value TEXT,
        type TEXT,
        FOREIGN KEY (id) REFERENCES nodes(id)
    );

    CREATE TABLE ways (
        id INTEGER PRIMARY KEY NOT NULL,
        user TEXT,
        uid INTEGER,
        version TEXT,
        changeset INTEGER,
        timestamp TEXT
    );

    CREATE TABLE ways_tags (
        id INTEGER NOT NULL,
        key TEXT NOT NULL,
        value TEXT NOT NULL,
        type TEXT,
        FOREIGN KEY (id) REFERENCES ways(id)
    );

    CREATE TABLE ways_nodes (
        id INTEGER NOT NULL,
        node_id INTEGER NOT NULL,
        position INTEGER NOT NULL,
        FOREIGN KEY (id) REFERENCES ways(id),
        FOREIGN KEY (node_id) REFERENCES nodes(id)
    );
    3.
    .import desktop/project/nodes.csv nodes
    .import desktop/project/nodes_tags.csv nodes_tags
    .import desktop/project/ways.csv ways
    .import desktop/project/ways_tags.csv ways_tags
    .import desktop/project/ways_nodes.csv ways_nodes
    

In [7]:
import sqlite3
db = sqlite3.connect("osm.db")
c = db.cursor()

In [8]:
QUERY = """
SELECT id, value, LENGTH(value) as total
FROM nodes_tags 
WHERE key ='postcode' and 
total != 7 and
total != 5;
"""
c.execute(QUERY)
rows = c.fetchall()

import pandas as pd
df=pd.DataFrame(rows)
print df

            0       1  2
0  1784228299  137070  6
1  1918198776  135070  6
2  2099140167     135  3
3  4132075644  107-36  6


In [9]:
QUERY = """
SELECT id, value, LENGTH(value) as total
FROM ways_tags 
WHERE key ='postcode' and 
total != 7 and
total != 5;
"""
c.execute(QUERY)
rows = c.fetchall()

import pandas as pd
df=pd.DataFrame(rows)
print df

           0       1  2
0  152764379  121839  6
1  323234688  122749  6
2  323234965  122749  6
3  323242342  122749  6
4  323243332  122749  6
5  420009951  122807  6


    Among 10 flawed postcode, I decied to correct 8 of them with missing '-' and drop 2 of them. Used 'update' and 'delete' to clean the dataset.

In [10]:
QUERY = """
UPDATE nodes_tags
SET value = '137-070'
WHERE id = 1784228299;
"""
c.execute(QUERY)

QUERY = """
UPDATE nodes_tags
SET value = '135-070'
WHERE id = 1918198776;
"""
c.execute(QUERY)

QUERY = """
UPDATE ways_tags
SET value = '121-839'
WHERE id = 152764379;
"""
c.execute(QUERY)

QUERY = """
UPDATE ways_tags
SET value = '122-749'
WHERE id = 323234688 or
id = 323234965 or
id = 323242342 or
id = 323243332;
"""
c.execute(QUERY)

QUERY = """
UPDATE ways_tags
SET value = '122-807'
WHERE id = 420009951;
"""
c.execute(QUERY)

QUERY = """
DELETE FROM nodes_tags
WHERE id = 2099140167 or
id = 4132075644;
"""
c.execute(QUERY)

QUERY = """
SELECT id, value, LENGTH(value) as total
FROM nodes_tags 
WHERE key ='postcode' and 
total != 7 and
total != 5;
"""
c.execute(QUERY)
rows = c.fetchall()

import pandas as pd
df=pd.DataFrame(rows)
print df

QUERY = """
SELECT id, value, LENGTH(value) as total
FROM ways_tags 
WHERE key ='postcode' and 
total != 7 and
total != 5;
"""
c.execute(QUERY)
rows = c.fetchall()

import pandas as pd
df=pd.DataFrame(rows)
print df

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []


## 3. Data Overview and Additional Ideas
---

### 1) File sizes
---
    central-seoul_south-korea.osm .. 79.5 MB
    osm.db ......................... 44.2 MB
    nodes.csv ...................... 27.8 MB
    nodes_tags.csv ................. 61 KB
    ways.csv ....................... 3.6 MB
    ways_tags.csv .................. 27 KB
    ways_nodes.cv .................. 10.1 MB 

### 2) Number of nodes
---

In [11]:
QUERY = """
SELECT COUNT(*) FROM nodes;
"""
c.execute(QUERY)
rows = c.fetchall()
print rows[0][0]

333753


### 3) Number of ways
---

In [12]:
QUERY = """
SELECT COUNT(*) FROM ways;
"""
c.execute(QUERY)
rows = c.fetchall()
print rows[0][0]

60047


### 4) Number of unique users
---

In [13]:
QUERY = """
SELECT COUNT(DISTINCT(e.uid))          
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;
"""
c.execute(QUERY)
rows = c.fetchall()
print rows[0][0]

982


### 5) Top 10 contributing users
---

In [14]:
QUERY = """
SELECT e.user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
GROUP BY e.user
ORDER BY num DESC
LIMIT 10;
"""
c.execute(QUERY)
rows = c.fetchall()

import pandas as pd
df=pd.DataFrame(rows)
print df

              0      1
0   maphunter36  47588
1      panhoong  35381
2       GPIOIPG  27520
3        dabeeo  26366
4          Exj　  22700
5  CitymapperHQ  19030
6          thbz  14088
7     bizarre07  13634
8       dabeeo2   9857
9     Stleamist   9282


### 6) Number of users appearing only once (having 1 post)
---

In [15]:
QUERY = """
SELECT COUNT(*) 
FROM
(SELECT e.user, COUNT(*) as num
 FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
 GROUP BY e.user
 HAVING num=1)  u;
"""
c.execute(QUERY)
rows = c.fetchall()
print rows[0][0]

312


## 4. Ideas
---

### 1) Numbers of universities in Seoul

    . Since I am a university student, I wanted to find out how many universities in Seoul. First, I tried to find university in 'name' key, but I soon realized it would be impossible to find name that ends with the word '대학교(which means university)' or they contain numerous kinds of primary schools, hospitals, museum etc. So I tried to query with the key 'name:ko' and 'name'en'. A bit surprising that there were approximately 55/59(Korean/English) universities in Seoul. Quite a lot! I could find my '고려대학교'(Korea University), but I couldn't find '연세대학교'(Yonsei University)...(Sorry)

In [16]:
QUERY = '''
SELECT key, count(key)
FROM nodes_tags
WHERE key = 'name:ko' and
value LIKE '%대학교' and
value != '대학교';
'''
#To find the list, change count(key) to value.
c.execute(QUERY)
rows = c.fetchall()
import pandas as pd
df=pd.DataFrame(rows)
print df

         0   1
0  name:ko  55


In [17]:
QUERY = '''
SELECT key, value
FROM nodes_tags
WHERE key = 'name:en' and
value LIKE '%korea%' and
value LIKE '%university' and
value != 'university';
'''
c.execute(QUERY)
rows = c.fetchall()
import pandas as pd
df=pd.DataFrame(rows)
print df

         0                                1
0  name:en      Korea Pollitek 1 University
1  name:en      Korea Pollitek 1 University
2  name:en            Korea Open University
3  name:en           Korea Cyber University
4  name:en                 Korea University
5  name:en            Korea Open University
6  name:en            Korea Open University
7  name:en  Korea Bencheojeongbo University


In [18]:
QUERY = '''
SELECT key, value
FROM nodes_tags
WHERE key = 'name:en' and
value LIKE '%yonsei%' and
value LIKE '%university' and
value != 'university';
'''
c.execute(QUERY)
rows = c.fetchall()
import pandas as pd
df=pd.DataFrame(rows)
print df

Empty DataFrame
Columns: []
Index: []


### 2) Top 10 Most Popular Amenities
---
    This list reflects the culture of eating out in Seoul...(Too many reastaurants)

In [19]:
QUERY = '''
SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='amenity'
GROUP BY value
ORDER BY num DESC
LIMIT 10;
'''
c.execute(QUERY)
rows = c.fetchall()
import pandas as pd
df=pd.DataFrame(rows)
print df


             0    1
0   restaurant  687
1         bank  674
2       school  514
3         cafe  415
4         fuel  323
5       police  286
6     townhall  279
7      library  260
8  post_office  245
9     hospital  237


### 3) Top 10 Popular Foods   

    Korean was the most popular food in Seoul(Of Course...)
---

In [20]:

QUERY = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='cuisine'
GROUP BY nodes_tags.value
ORDER BY num DESC
limit 10;
'''
c.execute(QUERY)
rows = c.fetchall()
import pandas as pd
df=pd.DataFrame(rows)
print df

             0   1
0       korean  40
1     regional  22
2      chinese  15
3      italian  15
4     japanese  15
5        pizza  14
6        asian   9
7  coffee_shop   5
8      mexican   5
9     american   4


### 4) Top 10 banks in Seoul
---
    Among the titled, '우리은행' has the most nodes. I thought there were too few banks named '농협', but I figured out the way they named was not just '농협' but like '관악농협신림지점앞' which state the place of bank with it.

In [21]:

QUERY = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='bank') i
    ON nodes_tags.id=i.id
where key = 'name' or
key = 'name:ko' or
key = 'name:en'
GROUP BY nodes_tags.value
ORDER BY num DESC
limit 10;
'''
c.execute(QUERY)
rows = c.fetchall()
import pandas as pd
df=pd.DataFrame(rows)
print df

             0   1
0         우리은행  88
1         신한은행  70
2         하나은행  65
3         국민은행  61
4         기업은행  32
5         외환은행  32
6           농협  31
7  한국스탠다드차타드은행  20
8         씨티은행  14
9        새마을금고  12


### 5) Top 10 coffeeshop in Seoul
---
    I think the number of coffeshop is way too small for Seoul. I guess Most of them are untitled. Top 1, 2, 4, 5were all Starbucks in different names(스타벅스 is Starbucks in Korean). I think these can be improved.

In [22]:

QUERY = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='cafe') i
    ON nodes_tags.id=i.id
where key = 'name' or
key = 'name:ko' or
key = 'name:en'
GROUP BY nodes_tags.value
ORDER BY num DESC
limit 10;
'''
c.execute(QUERY)
rows = c.fetchall()
import pandas as pd
df=pd.DataFrame(rows)
print df

                  0   1
0         Starbucks  24
1              스타벅스   8
2      Ediya Coffee   7
3         STARBUCKS   7
4  Starbucks Coffee   7
5              카페베네   6
6               커피빈   6
7    Dunkin' Donuts   5
8    Holly's Coffee   5
9          Pascucci   5


## 5. Conclusion

    The data of Seoul was quite well organized than expected though there are some incompleteness. Especially, the fact that most of the values were provided in Korean was quite impressive considering the popularity of OSM in Korea. Yet, there are much more things could be done. 
    1. Solution: Some errors found here(Some errors in postcodes, inconsistant name of Starbucks) could be fixed. 
        - Benifit
            1) Consistant names would contribute to more precise statictics.
            2) Precise postcodes can be use for functional purpose.
        - Anticipated Problems
            1) All the check should be done by human - would take lots of time and effort.
            2) It would be hard to check the absent postcodes.
    2. Solution: Nodes should have complete tags of 'name', 'name:ko' and 'name:en'(At least in two languages). 
        - Benifit
            1) Statistics would show consistant result, not varies by languages.
            2) Foreigners would not be confused to see mixed up names
        - Anticipated Problems
            1) All the check should be done by human - would take lots of time and effort.
            2) It would be hard to check the absent names.
    3. Solution: Mixed up between the nodes and ways could be fixed. 
        - Benifit
            1) Classification between nodes and ways would be more accurate.
        - Anticipated Problems
            1) There are too much nodes informations on ways(would take some time to fix all).
            2) There are some obscure nodes which can be seen as both nodes or ways.
    I think the GPS and node data in OSM are really valueable, so it would be crucial to clean and improve data in order to make useful use of this dataset.